# Peta ATM PRIME

In [1]:
def animated_loading():

    import sys, time

    chars = ["/", "—", "\\", "|"]

    for char in chars:

        sys.stdout.write('\rloading...'+char)
        time.sleep(.1)
        sys.stdout.flush()

In [2]:
def get_entry_from_tid(tid, keyword):
    
    import requests
    from bs4 import BeautifulSoup as bs
    html = requests.get('http://atmpro.bri.co.id/statusatm/viewatmdetail.pl?ATM_NUM='+tid).text
    soup = bs(html, "lxml").findAll('table', {'class':'fancy'})
    rows = soup[0].findAll('tr')
    try:
        for row in rows:
            
            if keyword in row.getText():
                tds = row.findAll('td')
                
        entry = tds[1].getText()
        
    except UnboundLocalError:
        
        entry = '???'
    
    return entry

In [3]:
def durasi_waktu(strDate):
    
    from datetime import datetime
    strDate = strDate.replace('_',' ')

    if strDate:
        if "-" in strDate:

            format1 = '%Y-%m-%d %H:%M:%S'
            strDate = strDate.replace(".0","")

        elif "/" in strDate:

            format1 = '%d/%m/%Y %H:%M'

        span = datetime.now() - datetime.strptime(strDate, format1)
        textDurasi = ':'.join(str(span).split('.')[:1]).replace('days','hari').replace('day','hari')
    else:
        textDurasi = "???"
        
    return textDurasi

In [4]:
keywords=[
    "Sukses Trx. Tunai",
    "MONEY LEFT IN DRAWER",
    "DISPENSER FAILURE",
    "COMMUNICATION",
    "ATM Out of Service",
    "SUPERVISORY MODE",
    "JOURNAL",
    "RECEIPT PRINTER",
    "SECURITY DEVICES (PINPAD)",
    "DOOR OPENNED",
    "PRESENTER ERROR",
    "REJECT BIN FULL",
    "CASH OUT",
    "CARD READER",
    "CASH LOW"
]

In [5]:
def html_formatter(k, v):
    
    if 'FAIL' in v:
        color = 'red'
    elif 'LOW' in v:
        color = 'grey'        
    else:
        color = 'green'
    
    html =  '<tr>'
    html += '<td><b style="color:'+color+'">'+k+'</b></td>'
    html += '</tr>'
    
    return html

def table_tag(html):
    
    return '<table>'+html+'</table>'

In [6]:
def link_atm(tid):
    link = '<a href=http://atmpro.bri.co.id/statusatm/viewatmdetail.pl?ATM_NUM='+str(tid)+' target=_blank>'+str(tid)+'</a>'
    return link

In [7]:
def html_tooltip(color, tid):
    tooltip = '<h2 style="color:+'+color+'">'+loc +' ('+ link_atm(tid)+')'+'</h2>' 
    return

In [8]:
def html_popup(tid):
    
    import requests
    from bs4 import BeautifulSoup as bs
    html = requests.get('http://atmpro.bri.co.id/statusatm/viewatmdetail.pl?ATM_NUM='+str(tid)).text
    soup = bs(html, "lxml").findAll('table', {'class':'fancy'})
    rows = soup[0].findAll('tr')
    try:
        html =''
        fail = 0
        cashlow = 0
        sukses_tartun = ''
        for keyword in keywords:
            
            for row in rows:
                if keyword in row.getText():
                    tds = row.findAll('td')
                
            k = tds[0].getText()
            v = tds[1].getText()
            # hanya yang 'FAIL' yang ditampilkan
            if 'FAIL' in v:
                html += html_formatter(k, v)                
                fail += 1
            if 'LOW' in v:
                k = k + ' LOW'
                html += html_formatter(k, v)                
            if k == 'CASH LOW':
                cashlow +=1
            if k == 'Sukses Trx. Tunai':
                sukses_tartun = durasi_waktu(v) +" yang lalu (" +(v)+")"
                
                
        if not fail:
            color = 'green'
            html='<tr><td><b style="color:green">EVERYTHING IS OKAY! <h2>&#128516;</h2></b></td></tr>'
        elif (fail==1) and (cashlow==1): 
            color = 'blue'
            html += '<h2 style="color:blue">&#128542;</h2>'
        else: 
            color = 'red'
            html += '<h2 style="color:red">&#128544;</h2>'
        if sukses_tartun:
            html += '<tr><td><br><b style="color:'+color+'">TARTUN '+sukses_tartun+'</b></td></tr>'
            
        popup = table_tag(html)

    except UnboundLocalError:
        
        popup = '???'
        color = 'blue'
    
    return popup, color


In [9]:
def show_atm_map(file_lokasi):
       
    import pandas as pd
    data = pd.read_csv(file_lokasi)
    data.columns = ['NO.', 'TID', 'LOC', 'LAT', 'LONG']
    LOC, LAT, LONG, TID = data.LOC, data.LAT, data.LONG, data.TID
    
    import folium

    m = folium.Map(location=[3.597031, 98.678513], zoom_start=7)

    for tid, loc, lat,long in zip(TID, LOC, LAT, LONG):
        animated_loading()
        popup,color = html_popup(tid)
        tooltip = '<h5><b style=color:'+color+'>'+loc +' ('+ (link_atm(tid))+')'+'</b></h5>'   
        popup = tooltip + popup
        icon = folium.Icon(color=color)
        folium.Marker([lat, long], popup=popup, tooltip=tooltip, icon=icon).add_to(m)
    
    return m


# TAMPILAN PETA LOKASI ATM PRIME

In [10]:
show_atm_map('lokasiatm.csv')

loading...|